# QOSF Cohort 7 Screening

## Task 4: Problem statement

Design a function that generates a random quantum circuit by considering as parameters the number of qubits, the number of depths, and the base of gates to be used. You could only use the quantum gates of 1 and 2 qubits. This implementation uses the Tequila quantum library.

> `def random_circuit (int:num_qubits, int:depth, list:basis_gates):`<br>
>>   `"""`<br>
>>    `Inputs:`<br>
>>>        `num_qubits : integer value that is the number of qubits.`<br>
>>>        `depth: integer value that is the depth for the random circuit.`<br>
>>>        `basis_gates : A list that contains the basis gates to generate the quantum circuit.`<br>
>>    `Return the quantum circuit`<br>
>>    `"""`<br>
>>    `# use a framework that works with quantum circuits, qiskit, cirq, pennylane, etc.`<br>
>>    `# print your quantum circuit`
    
The probelm statement ends here.

## Code usage details

The following code makes some basis assumptions as explained in the function description below. Some example usage is provided at the end of the file. Please execute all previous code cells before executing any cell in this Jupyter notebook.

In [1]:
import tequila as tq
from typing import Union, Optional
import numpy as np
from math import pi
from random import choice, sample

In [2]:
def random_circuit(num_qubits: int, depth: int, basis_gates: list[tq.QCircuit],
                   strict: Optional[bool] = True, powers: Optional[bool] = False,
                   back: Optional[str] = "qiskit") -> tq.QCircuit:

    """
    Inputs:
        num_qubits : integer value that is the number of qubits.
        depth: integer value that is the depth for the random circuit.
        basis_gates : A list that contains the basis gates to generate the quantum circuit.
    Return the quantum circuit
    
    Preconditions:
        1. All basis gates are either single qubit gates or double qubit gates with one control and one target
            This doesn't affect universality of the basis, but makes it easier to consider implementation in
            an abstract library such as Tequila. However, the current version of the code can still run without
            this assumption, as long as strict is set to False
        2. depth > 0 and num_qubits > 0 and len(basis_gates) > 0
        3. basis_gates contains at least one single qubit gate
        4. Identity operation not in basis (i.e. all basis operations non-trivial)
    """    
    basis_gates = [_get_gate(bg, powers=True) for bg in basis_gates]
    
    if strict: # Check preconditions (usually assumed to be given)
        # assert that depth and num_qubits are positive
        assert(depth > 0 and num_qubits > 0 and len(basis_gates) > 0)
        # assert that basis gates are all single circuit gates
        assert(all((len(bg.gates) == 1) and (bg.depth == 1) for bg in basis_gates))
        # assert that basis gates are all either single qubit or double qubit gates
        for bg in basis_gates:
            if bg.gates[0].control: assert(len(bg.qubits) == 2) # if controlled gate, then 2 qubits
            else: assert(len(bg.qubits) == 1) # else only 1 qubit
    
#     if powers: # If indicated, replace unparametrized gates with parameterized versions
#         for bg in basis_gates:
#             if not bg.gate[0].is_parametrized:     
#                 TODO
                
    singlebasis = [_get_gate(bg) for bg in basis_gates if len(_get_gate(bg).qubits) == 1] # all single qubit gates
    singlebasis += [tq.gates.Phase(target=0, angle=2*pi)] # add identity operation into basis also
    multibasis = [_get_gate(bg) for bg in basis_gates if len(_get_gate(bg).qubits) > 1] # all multi qubit gates
    circ = tq.QCircuit() # circ will act on qubits labelled over range(num_qubits)
    
    curr_depth = 0
    
    while curr_depth < depth:
        if not curr_depth:
            circ = _first_layer(circ, num_qubits, singlebasis, multibasis)
        else:
            if depth == 2:
                return _two_layers(circ, num_qubits, singlebasis, multibasis)
            circ = _later_layers(circ, num_qubits, singlebasis, multibasis)
        curr_depth += 1

    # print your quantum circuit
    tq.draw(circ)
    
    return circ


def _first_layer(circ: tq.QCircuit, num_qubits: int, 
                 singlebasis: list[tq.QCircuit], multibasis: list[tq.QCircuit],
                powers: bool = False) -> tq.QCircuit:
    """
        Construct the first layer of the quantum circuit using only non controlled gates.
    """
    gates = singlebasis + multibasis
    qubits = list(range(num_qubits))
    while qubits:
        # sample random gate
        while_cond = True
        while while_cond:
            gate = choice(gates)
            num = len(gate.qubits)
            while_cond = any(bool(g.control) for g in gate.gates) or bool(len(qubits) - num < 0)
        gate_qubits = sample(qubits, num)
        
        # update qubit mapping
        bit_mapping = dict()
        for j in range(num):
            bit_mapping.update({gate.qubits[j]: gate_qubits[j]})
        gate = gate.map_qubits(bit_mapping)
        
        # get qubit string for naming variables
        qubit_str = ""
        for q in gate_qubits:
            qubit_str += str(q)
        
        # update variable mapping
        params = gate.extract_variables()
        if params:
            param_mapping = dict()
            for k in range(len(params)):
                param_mapping.update({params[k]: "d2:"+qubit_str+":"+str(k)})
            gate = gate.map_variables(param_mapping)
        
        circ += gate
        # remove affected qubits from list of qubits untouched at this level
        for j in range(num):
            qubits.remove(gate_qubits[j])
    
    return circ


def _two_layers(circ, num_qubits, singlebasis, multibasis):
    """
        Return the second layer of a two-layer random circuit,
        assuming that the first layer has already been constructed.
    """
    qubits = list(range(num_qubits))    
    # add mostly multi-qubit gates if any multi qubit gates in basis
    if multibasis:
        while len(qubits) > 1:
            # get a random gate from the basis
            while_cond = True
            while while_cond:
                gate = choice(multibasis)
                num = len(gate.qubits)
                while_cond = bool(len(qubits) - num < 0)
            gate_qubits = sample(qubits, num)
        
            # get qubit string for naming variables
            qubit_str = ""
            for q in gate_qubits:
                qubit_str += str(q)
        
            # update qubit mapping and variable mapping
            bit_mapping = dict()
            for j in range(num):
                bit_mapping.update({gate.qubits[j]: gate_qubits[j]})
            params = gate.extract_variables()
            param_mapping = dict()
            for k in range(len(params)):
                param_mapping.update({params[k]: "d2:"+qubit_str+":"+str(k)})
            gate = gate.map_qubits(bit_mapping)
            gate = gate.map_variables(param_mapping)
        
            # add gate to circuit
            circ += gate
        
            # remove affected qubits from list of qubits untouched at this level
            for j in range(num):
                qubits.remove(gate_qubits[j])
    
    # now, add single-qubit gates to whatever qubits are still unused in this layer
    while qubits:
        # get a random gate from the basis
        gate = choice(singlebasis)
        num = len(gate.qubits)
        gate_qubits = sample(qubits, num)
        
        # get qubit string for naming variables
        qubit_str = ""
        for q in gate_qubits:
            qubit_str += str(q)
        
        # update qubit mapping
        bit_mapping = dict()
        for j in range(num):
            bit_mapping.update({gate.qubits[j]: gate_qubits[j]})
        gate = gate.map_qubits(bit_mapping)
        
        # update variable mapping
        params = gate.extract_variables()
        if params:
            param_mapping = dict()
            for k in range(len(params)):
                param_mapping.update({params[k]: "d2:"+qubit_str+":"+str(k)})
            gate = gate.map_variables(param_mapping)
    
        # add gate to circuit
        circ += gate
        
        # remove affected qubits from list of qubits untouched at this level
        for j in range(num):
            qubits.remove(gate_qubits[j])
    
    return circ # return circuit

        
def _later_layers(circ: tq.QCircuit, num_qubits: int, singlebasis, multibasis):
    """
        For all other depths > 2, no preference given to either single-qubit or multi-qubit gates
        after the first layer
    """
    qubits = list(range(num_qubits))
    gates = singlebasis + multibasis
    while qubits:
        # get a random gate from the basis
        while_cond = True
        while while_cond:
            gate = choice(gates)
            num = len(gate.qubits)
            while_cond = bool(len(qubits) - num < 0)
        gate_qubits = sample(qubits, num)
        
        # get qubit string for naming variables
        qubit_str = ""
        for q in gate_qubits:
            qubit_str += str(q)
        
        # update qubit mapping
        bit_mapping = dict()
        for j in range(num):
            bit_mapping.update({gate.qubits[j]: gate_qubits[j]})
        gate = gate.map_qubits(bit_mapping)
        
        # update variable mapping
        params = gate.extract_variables()
        if params:
            param_mapping = dict()
            for k in range(len(params)):
                param_mapping.update({params[k]: "d2:"+qubit_str+":"+str(k)})
            gate = gate.map_variables(param_mapping)
    
        # add gate to circuit
        circ += gate
        
        # remove affected qubits from list of qubits untouched at this level
        for j in range(num):
            qubits.remove(gate_qubits[j])
    return circ


def _get_gate(bg: Union[tq.QCircuit, str], var: Optional[str] = "",
             powers: bool = False) -> tq.QCircuit:
    if isinstance(bg, tq.QCircuit):
        if bg.is_fully_parametrized:
            old = bg.extract_variables()
            if old: return bg.map_variables({old[0]: var})
        return bg
    
    elif powers:
        # TODO: INCOMPLETE
    # could replace with switch case, but that might break downward compatibility
        if bg.lower() == "x":
            return tq.gates.Rx(target=0, angle=str(var+"rx"))
        elif bg.lower() == "y":
            return tq.gates.Ry(target=0, angle=str(var+"ry"))
        elif bg.lower() == "z":
            return _get_gate("Rz")
        elif bg.lower() == "h": return tq.gates.H(target=0)
        elif bg.lower() == "rx":
            return tq.gates.Rx(target=0, angle=str(var+"rx"))
        elif bg.lower() == "ry":
            return tq.gates.Ry(target=0, angle=str(var+"ry"))
        elif bg.lower() == "rz": 
            return tq.gates.Rz(target=0, angle=str(var+"rz"))
        elif bg.lower() == "cnot": 
            return tq.gates.Rx(target=1, control=1, angle=str(var+"rx"))
    else:
        ...

## Code examples and discussion

The basic function above prints the resulting circuit using Tequila's in-built circuit drawing tool. To get better circuit visualizations, specify a backend as shown in the cells below. There's obviously a lot of code repetition which can be easily cleaned up using helper functions.
0. I have given a sample usage in the following cell. Feel free to change the inputs as desired. If needed, reference the list of supported gates by visiting the gate.py file in Tequila's documentation.
1. I used the $2\pi$ Phase gate to simulate the Identity operation, as it allows us additional randomness in our circuits, by simply choosing not to apply any gate to a qubit at some layer. 
2. Additional modifications can be made by specifying a probability distribution over the random sampling from the basis of gates.
3. Another choice was made in deciding NOT to interpret unparametrized gates in the basis as their parametrized counterparts. In doing so, we retain an additional degree of flexibility which allows to specify both an unparametrixed gate and a parametrixed gate (e.g. both X and Rx) in the basis. This would be given by the (INCOMPLETE) `powers` option to the `random_circuit` function.
4. For the first layer, I chose to exclude any controlled gates since they will not give any interesting results (assuming we start out in the zero state, like in Tequila). For depth-2 circuits, I chose to prioritize multi-qubit gates for the second layer. For all other cases, both multi-qubit and single-qubit, and controlled and non-controlled, gates were considered equally.

There are two more convenience choices which I would like to address. These choices were made in line to Tequila's founding principles, and the subsequent decisions to try implementing some of these convenience functions also imitates how Tequila decided to include some convenience functions upon user feedback.

5. The input list `basis_gates` must currently be given as a list of Tequila circuits / gates. However, the extension to providing a list of strings or a Union of both strings and Tequila objects is straightforward. In the code above, I have specified an obvious way in how to go about doing so. Currently, supported strings are X, Y, Z, Rx, Ry, Rz, H, and CNOT, but no other multi-qubit or controlled gates.
6. Alternatively, another method for doing this would be to provide a parser which would parse strings into Tequila objects only once at the very start. 

In [4]:
basis = [tq.gates.X(0), tq.gates.Z(0), tq.gates.X(target=0, control=1)]
basis += [tq.gates.Rx(target=0,angle="a"), tq.gates.Rx(target=0, angle="b"), tq.gates.Rx(target=0, control=1, angle="c")]
basis += ["X", tq.gates.Rx(target=0,angle="a"), tq.gates.Rx(target=0, angle="b"), tq.gates.Rx(target=0, control=1, angle="c")]
# basis += [tq.gates.Rp("X(0)Y(0)Z(0)", angle="phi")] # use only if strict is False !

depth = 2
num_qubits = 2

circ = random_circuit(depth, num_qubits, basis)

print(circ.depth == depth)
print(circ.qubits == list(range(num_qubits)))

tq.draw(circ)
# tq.draw(circ, backend="qiskit") # Since Qiskit supports different gates, upon compiling, the depth of the circuit may differ

True
True
0: ───X^(0.318309886183791*f((d2:0:0,))_0)───@───────────────────────────────────────
                                             │
1: ───Z──────────────────────────────────────X^(0.318309886183791*f((d2:10:0,))_1)───


''